# Desafio Data Science Intelivix
  Por: [Paulo Romeira](https://github.com/pauloromeira)

In [1]:
# Classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Pré-processamento
# from sklearn.preprocessing import StandardScaler
# from sklearn import preprocessing

# Treinamento
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# Outros
import pandas as pd
from itertools import count
from IPython.display import display

# Dataset
# from sklearn.datasets import load_iris

# Random state: seed para que o notebook possa ser reproduzido.
R_STATE = 10

# Dataset

In [2]:
# Easy way :)
# iris = load_iris()
# X, y = iris.data, iris.target

header_names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
ds = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                 header=None,
                 names=header_names
                )
# Mapeia as classes para números
class_map = { c:i for c, i in zip(ds['class'].unique(), count()) }
ds['target'] = ds['class'].map(class_map)

# Usaremos apenas as variáveis X e y daqui em diante
X, y = ds[header_names[:-1]], ds['target']

print('Map: {0}'.format(class_map))
display(ds.groupby('class').first())

Map: {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


,sepal length,sepal width,petal length,petal width,target
class,,,,,
Iris-setosa,5.1,3.5,1.4,0.2,0
Iris-versicolor,7.0,3.2,4.7,1.4,1
Iris-virginica,6.3,3.3,6.0,2.5,2


# Classificadores e intervalos de hiperparâmetros
* Obs: é possível acrescentar novos classificadores nesta seção, sem a necessidade de alterar o resto do código.

In [3]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression()
]

# Apenas por conveniência
classifiers_names = ['KNN', 'SVC', 'L. Regression']

# Definindo os valores possíveis dos hiperparâmetros dos classificadores
c_range = list(pow(10,n) for n in range(-4,4))
params = [
    { 'n_neighbors': list(range(1,20)), 'weights': ['uniform', 'distance'] }, # KNN
    { 'kernel':('linear', 'rbf'), 'C': c_range }, # SVC
    { 'C': c_range } # L. Regression
]

## Otimização de hiperparâmetros

In [4]:
# Os dados são divididos em 3 (33% para teste) e cada parâmetro será testado 3 vezes - um para cada "fold".
cv = KFold(n_splits=3, shuffle=True, random_state=R_STATE)

for classifier, name, param in zip(classifiers, classifiers_names, params):
    # Aqui, cada classificador é testado em cross-validation pelo GridSearch
    # para cada combinação possível de parâmetros.
    grid = GridSearchCV(classifier, param, scoring='accuracy', cv=cv).fit(X, y)

    # Setando parâmetros achados no gridsearch
    # Obs: descartamos o 'best_estimator_' aqui, pois avaliaremos outras métricas.

    classifier.set_params(**grid.best_params_)
    
    print('{0}: {1} - {2:.2%}'.format(name, grid.best_params_, grid.best_score_))

KNN: {'n_neighbors': 4, 'weights': 'distance'} - 96.00%
SVC: {'C': 1, 'kernel': 'linear'} - 98.67%
L. Regression: {'C': 100} - 96.00%


> Obs: na busca pelos melhores parâmetros acima, o classificador com maior média de acertos no cross-validation foi o SVC, com 98.67%.

## Treinamento dos classificadores

In [5]:
# Dividindo as amostras em 66% treino e 34% testes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=R_STATE)

predicted = []
accuracy_scores = []
confusion_matrices = []

for classifier, name in zip(classifiers, classifiers_names):
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    predicted.append(y_pred)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))
    
    print("{0}: {1:.2%}".
          format(name, accuracy_scores[-1]))

KNN: 94.12%
SVC: 96.08%
L. Regression: 98.04%


# Análise dos classificadores

In [6]:
for name, conf, score in zip(classifiers_names, confusion_matrices, accuracy_scores):
    print('{0}: {1:.2%}\n'.format(name, score), conf)

KNN: 94.12%
 [[15  0  0]
 [ 0 17  3]
 [ 0  0 16]]
SVC: 96.08%
 [[15  0  0]
 [ 0 18  2]
 [ 0  0 16]]
L. Regression: 98.04%
 [[15  0  0]
 [ 0 19  1]
 [ 0  0 16]]
